# Intermediate Python

A brief discussion of the following topics:

- dictionaries and iterating
- functions: arguments / parameters and variable scope
- lambdas
- error handling

All of these are particularly relevant for our next topic, Pandas.

## Dictionaries Revisited

### Creation and Access

In [ ]:
scores = {'Alice': 85, 'Bob': 92, 'Charlie': 98}

In [ ]:
# key must exist to be accessed without error
scores['Alice']

In [ ]:
scores['Dan']

A common pattern for creating / updating keys uses `if ... else`:

In [ ]:
# adjust score if record exists, otherwise create record
if 'Dan' not in scores:
    scores['Dan'] = 0    # create new kv pair
else:
    scores['Dan'] += 10  # increase existing value

print(scores['Dan'])

The `get` method provides a safe way to access values without risking KeyError exceptions. It takes two arguments, the key and a default value if the key does not exist.

In [ ]:
# Same operation, much cleaner!
scores['Eddie'] = scores.get('Eddie', 0) + 10   # Creates with 10
scores['Alice'] = scores.get('Alice', 0) + 10   # Updates to 95

print(scores)

Another example of how `get` is commonly used.

In [ ]:
word_count = {}
text = "the cat and the dog and the bird"

# Without get(): requires if/else
for word in text.split():
    if word in word_count:
        word_count[word] += 1
    else:
        word_count[word] = 1

# With get(): single line per word
word_count = {}
for word in text.split():
    word_count[word] = word_count.get(word, 0) + 1

### Iteration

Nested dictionaries are commonly used.

In [ ]:
# Sample data dictionary (similar to what you'll see in data analysis)
student_scores = {
    'Alice': {'math': 92, 'science': 88, 'english': 95},
    'Bob': {'math': 78, 'science': 92, 'english': 81},
    'Charlie': {'math': 95, 'science': 79, 'english': 88}
}

Dictionaries can be iterated over by key, value, or item (ke-value pair):

In [ ]:
# Iterating through keys
for student in student_scores.keys():
    print(f"{student}: {student_scores[student]}")

In [ ]:
# Keys is the default, the method isn't required
for student in student_scores.keys():
    print(f"{student}: {student_scores[student]}")

In [ ]:
# Iterating through values only
all_math_scores = []
for scores in student_scores.values():
    all_math_scores.append(scores['math'])

print(all_math_scores)

In [ ]:
# Iterating through items (kv pairs)
for student, scores in student_scores.items():
    avg_score = sum(scores.values()) / len(scores)
    print(f"{student}'s average: {avg_score:.1f}")

### Comprehensions

Dictionaries also support comprehensions, using equivalent syntax:

```python
{k: v for element in iterable}
```

Simple examples:

In [ ]:
# Basic dictionary comprehension
numbers = [1, 2, 3, 4, 5]
squares = {n: n**2 for n in numbers}
print(squares)

In [ ]:
# From existing dictionary
scores = {'Alice': 85, 'Bob': 92, 'Charlie': 78}
curved = {name: score + 5 for name, score in scores.items()}
print(curved)

In [ ]:
# With filtering
passing = {name: score for name, score in scores.items() if score >= 80}
print(passing)

This pattern is commonly used to create new dicts from calculated and/or filtered values.

In [ ]:
# Create a new dictionary with calculated values
averages = {
    student: sum(scores.values()) / len(scores)
    for student, scores in student_scores.items()
}

print(averages)

In [ ]:
# Filter dictionary based on condition
high_performers = {
    student: scores 
    for student, scores in student_scores.items()
    if sum(scores.values()) / len(scores) > 85
}

print(high_performers)

## Functions: Arguments, Parameters, and Variable Scope

### Arguments and Parameters

Remember this distinction:

- argument refers to the items in a function **call**
- parameter refers to the items in a function **definition**

When a function is called, each argument can be specified either by position or keyword. It is up to the user.

Positional arguments must be provided in the exact order defined in the function. All positional arguments must be provided unless a default is defined in the function definition. Parameters with defaults need not be specified in the function call.

In [ ]:
# parameters defined with function
def greet(name, age, city='Auburn'):
    return f"{name}, {age}, from {city}"

# arguments passed by position here - order matters
greet("Alice", 25, "NYC")

In [ ]:
greet("Bob", 40)

In [ ]:
# order matters!
greet(55, 'Charlie', 'Tuskaloosa')

Keyword arguments are specified by name, order does not matter.

In [ ]:
greet(age=25, city="NYC", name="Alice")  # Any order

If positional and keyword arguments are mixed, the positional arguments must come first.

In [ ]:
greet("Alice", city="NYC", age=25)  # Mixed, positional first

Parameters with defaults must come after parameters without them. **Defaults are evaluated once, at the time the function is defined.** There are important implications of this, as we will demonstrate later!

### `args` and `kwargs`

The syntax `*args` and `**kwargs`, when used in a function definition, collects all the positional (`args`) and keyword (`kwargs`) in a tuple or dictionary, respectively. The names `args` and `kwargs` are just variables and can be anything, but are commonly used in Python. In the following example, `scores` and `options` are used instead.

In [ ]:
def analyze_scores(student_name, *scores, **options):
    """
    Demonstrates different parameter types
    - student_name: positional argument (required)
    - *scores: variable positional arguments (tuple)
    - **options: keyword arguments (dictionary)
    """
    verbose = options.get('verbose', False)
    include_median = options.get('include_median', True)
    
    if verbose:
        print(f"Analyzing {student_name}'s scores: {scores}")
    
    result = {
        'mean': sum(scores) / len(scores) if scores else 0,
        'max': max(scores) if scores else 0,
        'min': min(scores) if scores else 0
    }
    
    if include_median:
        sorted_scores = sorted(scores)
        n = len(sorted_scores)
        result['median'] = (sorted_scores[n//2] if n % 2 
                           else (sorted_scores[n//2-1] + sorted_scores[n//2])/2)
    
    return result

# Various ways to call the function
result1 = analyze_scores('Alice', 92, 88, 95)
result2 = analyze_scores('Bob', 78, 92, 81, verbose=True)
result3 = analyze_scores('Charlie', 95, 79, 88, verbose=True, include_median=False)

### Default Mutable Arguments (Common Pitfall)

In [ ]:
# WRONG - mutable default argument
def add_score_wrong(score, scores_list=[]):
    scores_list.append(score)
    return scores_list

In [ ]:
# CORRECT - use None as default
def add_score_correct(score, scores_list=None):
    if scores_list is None:
        scores_list = []
    scores_list.append(score)
    return scores_list

### Variable Scope and the LEGB Rule
Python resolves variables using LEGB order: Local → Enclosing → Global → Built-in

We will mostly ignore enclosing scope (more advanced), though it will come into play when we talk about decorators.

In [ ]:
global_threshold = 80  # Global scope

def create_grader(curve=5):  # Enclosing scope
    
    def grade(score):  # Local scope
        # Python looks for variables in LEGB order
        adjusted = score + curve  # 'curve' from enclosing scope
        
        if adjusted >= global_threshold:  # 'global_threshold' from global scope
            return 'Pass'
        return 'Fail'
    
    return grade

In [ ]:
count = 0  # Global variable

def process_data(data):
    global count  # Explicitly declare we want to modify global
    
    total = 0  # Local variable
    
    def update_stats(value):
        nonlocal total  # Modify enclosing scope variable
        global count
        
        total += value
        count += 1
    
    for item in data:
        update_stats(item)
    
    return total / len(data) if data else 0

average = process_data([10, 20, 30])
print(f"Processed {count} items")  # count was modified globally

## Lambda Functions

Lambdas are anonymous functions perfect for simple operations:

In [ ]:
# Traditional function
def square(x):
    return x ** 2

# Equivalent lambda
square_lambda = lambda x: x ** 2

# Multiple parameters
multiply = lambda x, y: x * y

# Conditional logic in lambda
grade_pass = lambda score: 'Pass' if score >= 60 else 'Fail'

They are particularly useful with built-in functions and methods like `sorted()` and `.sort()`:

In [ ]:
scores = [78, 92, 45, 88, 67, 95, 81]

# Map: Transform each element
curved_scores = list(map(lambda x: min(x + 10, 100), scores))

# Filter: Select elements meeting criteria
passing_scores = list(filter(lambda x: x >= 70, scores))

# Sorted: Custom sorting
students = [
    {'name': 'Alice', 'score': 92},
    {'name': 'Bob', 'score': 78},
    {'name': 'Charlie', 'score': 95}
]

# Sort by score
by_score = sorted(students, key=lambda s: s['score'], reverse=True)

# Sort by name length, then alphabetically
by_name_complexity = sorted(students, 
                           key=lambda s: (len(s['name']), s['name']))